In [12]:
# please install below pip package in order to use this code snippet
# Packages : pycontrol

In [13]:
# Need to run below in case you are using python version 2.7.6 and above

In [14]:
import urllib2
import ssl
from suds import transport
from suds.client import Client
from suds.xsd.doctor import Import, ImportDoctor
from pycontrol import pycontrol

IMP = Import('http://schemas.xmlsoap.org/soap/encoding/')
DOCTOR = ImportDoctor(IMP)
ICONTROL_URI = '/iControl/iControlPortal.cgi'
SESSION_WSDL = 'System.Session'

class HTTPSUnVerifiedCertTransport(transport.https.HttpAuthenticated):
    def __init__(self, *args, **kwargs):
        transport.https.HttpAuthenticated.__init__(self, *args, **kwargs)
    def u2handlers(self):
        handlers = []
        handlers.append(urllib2.ProxyHandler(self.proxy))
        handlers.append(urllib2.HTTPBasicAuthHandler(self.pm))
        # python ssl Context support - PEP 0466
        if hasattr(ssl, '_create_unverified_context'):
            ssl_context = ssl._create_unverified_context()
            handlers.append(urllib2.HTTPSHandler(context=ssl_context))
        else:
            handlers.append(urllib2.HTTPSHandler())
        return handlers

def new_get_suds_client(self, url, **kw):
        if not url.startswith('https'):
            t = transport.http.HttpAuthenticated(username=self.username,
                                                 password=self.password)
            c = Client(url, transport=t, username=self.username,
                         password=self.password, doctor=DOCTOR, **kw)
        else:
            t = HTTPSUnVerifiedCertTransport(username=self.username,
                                             password=self.password)
            c = Client(url, transport=t, username=self.username,
                         password=self.password, doctor=DOCTOR, **kw)
        return c

pycontrol.BIGIP._get_suds_client = new_get_suds_client
print "setup complete"

setup complete


In [15]:
# Below we are building dictionary of all virtual servers, pools and
# their nodes.
# call this method with your creds to see how it works

In [16]:
def import_f5_data(host, user, password):
    
    # Instantiate F5 Client
    b = None
    try:
        b = pc.BIGIP(
            hostname = host,
            username = user,
            password = password,
            fromurl = True,
            wsdls = ['LocalLB.Pool','LocalLB.VirtualServer']
        )
    except Exception:
        client.close()
        return None

    virtual_servers = b.LocalLB.VirtualServer.get_list()

    for server in virtual_servers:
        data={}
        try:
            data['f5_server'] = host
            data['datetime'] = dt.datetime.utcnow()

            # Virtual Server data
            data['virtual_server'] = server
            virtual_server = []
            virt = {}
            index = 0
            destinations = b.LocalLB.VirtualServer.get_destination_v2([server])
            for destination in destinations:
                virt = {}
                virt['ip'] = destination['address']
                virt['port'] = destination['port']
                virtual_server.append(virt)
                index += 1
            data['vip'] = virtual_server

            # Pool data
            poolname = server+"_pool"
            data['pool'] = poolname
            lb_method = b.LocalLB.Pool.get_lb_method([poolname])
            members = b.LocalLB.Pool.get_member([poolname])
            monitors = b.LocalLB.Pool.get_monitor_association([poolname])
            try:
                data['lb_method']=lb_method
            except:
                pass
            # Monitor data
            monitor = []
            for mtr in monitors[0][1][2]:
                monitor.append(mtr)
            data['monitor'] = monitor

            # Member data
            member = []
            mems = {}
            for mem in members[0]:
                mems = {}
                mems['server'] = mem['address']
                mems['port'] = mem['port']
                member.append(mems)
            data['member'] = member
        except Exception:
            pass
        print data

In [17]:
# Similar method for global traffic manager

In [18]:
def import_gtm_data(host, user, password):
    b = None
    try:
        b = pycontrol.BIGIP(
            hostname = host,
            username = user,
            password = password,
            fromurl = True,
            wsdls = ['GlobalLB.WideIP','GlobalLB.Pool','GlobalLB.PoolMember']
        )
    except Exception:
        client.close()
        return None
    wide_ips = b.GlobalLB.WideIP.get_list()
    pools = b.GlobalLB.Pool.get_list()
    data = {}
    for each in pools:
        data['pool'] = str(each)
        members = b.GlobalLB.Pool.get_member([each])
        memsarr = []
        for mem in members[0]:
            mems = {}
            mems['server'] = mem[0][0]
            mems['port'] = mem[0][1]
            memsarr.append(mems)
        data['members'] = memsarr
        print data
   